## Pydantic - Data Validation

In [1]:
from pydantic import BaseModel

In [2]:
class User(BaseModel):
    id: int    #Required
    name: str = 'Jane Doe'  #Not Required

In [3]:
user = User(id=100)
user

User(id=100, name='Jane Doe')

In [4]:
print(user.id)
print(user.name)

100
Jane Doe


In [5]:
user_2 = User(id='102') #Converts to integer automatically if string is number
user_2

User(id=102, name='Jane Doe')

#### ValidationError - Exception

In [6]:
user_3 = User(id='mk145') # This will fail as the string containg alpha characters
user_3

ValidationError: 1 validation error for User
id
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='mk145', input_type=str]
    For further information visit https://errors.pydantic.dev/2.5/v/int_parsing

#### Handling Validation Error

In [7]:
from pydantic import ValidationError

In [8]:
try:
    user_3 = User(id='mk145') # This will fail as the string containg alpha characters
    user_3
except ValidationError as ve:
    print(ve)

1 validation error for User
id
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='mk145', input_type=str]
    For further information visit https://errors.pydantic.dev/2.5/v/int_parsing


#### Creating model instace from dictonary object

In [9]:
user_dict = {
    "id": 101,
    "name": "Mohan"
}

user = User(**user_dict) # Keyword Arguments
user

User(id=101, name='Mohan')

In [10]:
user.id

101

In [11]:
user_2_dict = {
    "id": "Some data",
    "name": 144
}

try:
    user2 = User(**user_2_dict)
    user2
except ValidationError as ve:
    print(ve)

2 validation errors for User
id
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='Some data', input_type=str]
    For further information visit https://errors.pydantic.dev/2.5/v/int_parsing
name
  Input should be a valid string [type=string_type, input_value=144, input_type=int]
    For further information visit https://errors.pydantic.dev/2.5/v/string_type


#### modeldump() - Converts model instance to dictonary object

In [12]:
user_3 = User(id = 104, name = "Paul")
user_3

User(id=104, name='Paul')

In [13]:
user_dict = user_3.model_dump()
user_dict

{'id': 104, 'name': 'Paul'}

### Nested Models

In [14]:
from typing import List
from pydantic import PositiveInt

In [15]:
class Address(BaseModel):
    street: str
    city: str
    state: str
    pincode: int | str
    country: str = "India"

class Customer(BaseModel):
    id: int
    name: str
    age: PositiveInt
    phone_numbers: List[str]
    address: Address

In [17]:
myCustomerDict = {
    "id": 101,
    "name": "Nikola",
    "age": 45,
    "phone_numbers": ['+12 345678', '+12 457890'],
    "address": {
        "street": "AMT St.",
        "city": "Los Santos",
        "state": "California",
        "pincode": 345987,
        "country": "USA"
    }
}

In [18]:
customer = Customer(**myCustomerDict)

In [19]:
customer

Customer(id=101, name='Nikola', age=45, phone_numbers=['+12 345678', '+12 457890'], address=Address(street='AMT St.', city='Los Santos', state='California', pincode=345987, country='USA'))

In [20]:
myCustomerDict = {
    "id": 101,
    "name": "Nikola",
    "age": -45,
    "phone_numbers": ['+12 345678', '+12 457890'],
    "address": {
        "city": "Los Santos",
        "state": "California",
        "pincode": 345987
    }
}

try:
    customer = Customer(**myCustomerDict)
    print(customer)
except ValidationError as ve:
    print(ve)

2 validation errors for Customer
age
  Input should be greater than 0 [type=greater_than, input_value=-45, input_type=int]
    For further information visit https://errors.pydantic.dev/2.5/v/greater_than
address.street
  Field required [type=missing, input_value={'city': 'Los Santos', 's...nia', 'pincode': 345987}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/missing


#### Helper Functions
1. model_validate() -> Takes object or dict rather than keyword arguments for validation 
2. model_validate_json() -> Takes string and parse it as JSON for validation

In [21]:
from pydantic import BaseModel, ValidationError

In [22]:
class User(BaseModel):
    id: int
    name: str = 'John Doe'

In [23]:
user = User.model_validate({'id': 123, 'name': 'James'})
user

User(id=123, name='James')

In [24]:
try:
    user = User.model_validate([123, "John"])
    print(user)
except ValidationError as ve:
    print(ve)

1 validation error for User
  Input should be a valid dictionary or instance of User [type=model_type, input_value=[123, 'John'], input_type=list]
    For further information visit https://errors.pydantic.dev/2.5/v/model_type


In [25]:
user = User.model_validate_json('{"id": 123, "name": "James"}')
user

User(id=123, name='James')

In [26]:
try:
    user = User.model_validate_json('{"name": 456}')
    print(user)
except ValidationError as ve:
    print(ve)

2 validation errors for User
id
  Field required [type=missing, input_value={'name': 456}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/missing
name
  Input should be a valid string [type=string_type, input_value=456, input_type=int]
    For further information visit https://errors.pydantic.dev/2.5/v/string_type


#### model_construct() -> Create a new instance of the model without validation

This is helpful in scenarios where the instance needs to be created but validation is not required

In [27]:
class User(BaseModel):
    id: int
    age: int
    name: str = 'John Doe'

In [28]:
validUserDict = {
    "id": 101,
    "age": 43,
    "name": "Paul"
}

invalidUserDict = {
    "id": 102,
    "name": "Tina"
}

In [29]:
valid_user = User(**validUserDict)
valid_user

User(id=101, age=43, name='Paul')

In [30]:
valid_user_data = valid_user.model_dump()

In [31]:
fields_set = valid_user.model_fields_set
fields_set

{'age', 'id', 'name'}

In [32]:
valid_user_new = User.model_construct(_fields_set=fields_set, **valid_user_data)
valid_user_new

User(id=101, age=43, name='Paul')

In [33]:
invalid_user = User.model_construct(**invalidUserDict)  #invalidUserDict is not a valid data but the instace is created as validation is skipped 
invalid_user

User(id=102, name='Tina')

In [34]:
empty_user = User.model_construct()  #It even creates instance without passing the values
empty_user

User(name='John Doe')

#### Faux immutability - Forcing the feilds to be immuatable

In [35]:
from pydantic import BaseModel, ConfigDict, ValidationError

In [36]:
class MutableModel(BaseModel):
    model_config = ConfigDict(frozen=False) #Default

    f1: str
    f2: int

In [37]:
im = MutableModel(f1="Field 1", f2=100)
im

MutableModel(f1='Field 1', f2=100)

In [38]:
im.f1 = "Mutable Field"
im.f1

'Mutable Field'

In [39]:
class ImmutableModel(BaseModel):
    model_config = ConfigDict(frozen=True)

    f1: str
    f2: int

In [40]:
im = ImmutableModel(f1="Field 1", f2=100)
im

ImmutableModel(f1='Field 1', f2=100)

In [41]:
try:
    im.f1 = "Immutable Field"
except ValidationError as ve:
    print(ve)

1 validation error for ImmutableModel
f1
  Instance is frozen [type=frozen_instance, input_value='Immutable Field', input_type=str]
    For further information visit https://errors.pydantic.dev/2.5/v/frozen_instance


#### Extra Fields

In [43]:
class Model(BaseModel):
    f1: str

In [44]:
m = Model(f1="Field 1", f2="Field 2") # f2 is not defined in model but no error will be thrown by default
m

Model(f1='Field 1')

In [47]:
class Model(BaseModel):
    model_config = ConfigDict(extra='forbid')  # Throws error if any extra fields are passed while creating Model Instance

    f1: str

In [48]:
try:
    m = Model(f1="Field 1", f2="Field 2")
    print(m)
except ValidationError as ve:
    print(ve)

1 validation error for Model
f2
  Extra inputs are not permitted [type=extra_forbidden, input_value='Field 2', input_type=str]
    For further information visit https://errors.pydantic.dev/2.5/v/extra_forbidden


In [49]:
class Model(BaseModel):
    model_config = ConfigDict(extra='allow')  # Allows the undefined fields

    f1: str

In [50]:
m = Model(f1="Field 1", f2="Field 2")
m.__pydantic_extra__  # Return extra fields which are not defined in Model

{'f2': 'Field 2'}

### Field
This function is used to configure the feild definations of the Model

In [51]:
from pydantic import BaseModel, Field, ValidationError
from uuid import uuid4
from decimal import Decimal

In [52]:
class User(BaseModel):
    id: str = Field(default_factory=lambda: uuid4().hex, description="Autogenerated User ID") #Dynamic Default Values
    name: str = Field(default="John", description="User Name,") #Static Default Values
    age: int = Field(..., description="User Age in Years") # Elipsis(...) indiactes theat field is required
    email: str = Field(alias="contact", description="Email ID") #Contact is the alias name for email field

In [53]:
user = User(age=34, contact="john@testmail.com")
user

User(id='2b4405753ef0424581a6e5f9a070cfb2', name='John', age=34, email='john@testmail.com')

In [55]:
class NumericModel(BaseModel):
    positive: int = Field(gt=0)
    non_negative: int = Field(ge=0)
    negative: int = Field(lt=0)
    non_positive: int = Field(le=0)
    even: int = Field(multiple_of=2)
    multiples_of_10: int = Field(multiple_of=10)
    inf_nan_val: float = Field(allow_inf_nan=True)
    non_inf_nan_val: float = Field(allow_inf_nan=False)

    float_num: Decimal = Field(max_digits=5, decimal_places=2) #Maximum total digits must be 5 and maximum decimal place digits must be 2

In [59]:
numModel = NumericModel(
    positive=1,
    non_negative=0,
    negative=-1,
    non_positive=0,
    even=2,
    multiples_of_10=40,
    inf_nan_val=float('nan'),
    non_inf_nan_val=float(456.24),
    float_num=341.75
)

numModel

NumericModel(positive=1, non_negative=0, negative=-1, non_positive=0, even=2, multiples_of_10=40, inf_nan_val=nan, non_inf_nan_val=456.24, float_num=Decimal('341.75'))

In [62]:
class StringModel(BaseModel):
    gender: str = Field(min_length=4, max_length=6)
    email: str = Field(pattern=r'^.+@.+.com$')

In [63]:
strModel = StringModel(
    gender="Male",
    email="john@test.com"
)
strModel

StringModel(gender='Male', email='john@test.com')

In [64]:
class StrictModel(BaseModel):
    num1: int = Field(..., strict=False)
    num2: int = Field(..., strict=True)

In [67]:
model = StrictModel(num1="234", num2=345)
model

StrictModel(num1=234, num2=345)

In [68]:
type(model.num1)

int

In [69]:
class FrozenModel(BaseModel):
    num1: int = Field(..., frozen=False)
    num2: int = Field(..., frozen=True)

In [70]:
model = FrozenModel(num1=10, num2=12)
model

FrozenModel(num1=10, num2=12)

In [71]:
model.num1 = 14
model

FrozenModel(num1=14, num2=12)

In [72]:
try:
    model.num2 = 16
except ValidationError as ve:
    print(ve)

1 validation error for FrozenModel
num2
  Field is frozen [type=frozen_field, input_value=16, input_type=int]
    For further information visit https://errors.pydantic.dev/2.5/v/frozen_field
